In [14]:
import pandas as pd

names=pd.read_csv('/Users/hubertcheung/Downloads/temp.csv')
names=names.dropna()
nameparse=names['name'].str.split()

nameparse = nameparse[nameparse.apply(lambda x: (isinstance(x, list) and len(x)>0))]
nameparse = nameparse[nameparse.apply(lambda x: 'LTD' not in x)]
nameparse

1          [LEE, WAI, HONG, &, NGAN, TIK, HAN, NDIYA, (JT)]
2                                         [CHAN, WAI, LUEN]
4                                    [LI, YEE, LIN, ELAINE]
5                  [CHU, MING, WAH, &, YU, LAI, PING, (JT)]
6                  [CHU, PO, LUN, &, LAU, SHUK, MING, (JT)]
                                 ...                       
2261121            [LIU, WING, KAI, &, LO, LAI, YING, (JT)]
2261122        [LEUNG, TIM, &, LEUNG, CHIM, YAN, IDY, (JT)]
2261123         [MOK, KANG, MING, &, LEE, CHI, CHING, (JT)]
2261124                                 [HAU, SIU, PO, 1/2]
2261125           [NG, KATHERINE, &, LI, CHOR, KEUNG, (JT)]
Name: name, Length: 2030377, dtype: object

In [123]:
with open('canton.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

canton = [line.strip() for line in lines]

In [124]:
with open('pinyin.txt','r',encoding='utf-8') as file:
    lines=file.readlines()

pinyin = [line.strip() for line in lines]

In [120]:
with open('last_name_pinyin.txt','r',encoding='utf-8') as file:
    lines=file.readlines()
    
last_name_pinyin = [line.strip() for line in lines]

In [121]:
with open('last_name_canton.txt','r',encoding='utf-8') as file:
    lines=file.readlines()
    
last_name_canton = [line.strip() for line in lines]

In [116]:
test_HK = onename[onename.apply(lambda x: x[0] in last_name_canton)]

In [117]:
test_ML = onename[onename.apply(lambda x: x[0] in last_name_pinyin)]

In [118]:
resid = onename[onename.apply(lambda x: x[0] not in last_name_canton and x[0] not in last_name_pinyin)]
resid.to_csv('resid.csv')

In [6]:
pinyin_dual = [a+b for a in pinyin for b in pinyin]

In [15]:
onename = nameparse[nameparse.astype(str).apply(lambda x: '&' not in x)]
#Those with only one element are weird and should be picked out
singleelement = onename[onename.apply(lambda x:len(x)==1)]
onename = onename[onename.apply(lambda x:len(x)!=1)]

In [201]:
df = pd.DataFrame(nameparse)

In [7]:
HKname = onename[onename.apply(lambda x: x[0] in canton and x[1] in canton)]
print(len(HKname))
HKname_v1 = onename[onename.apply(lambda x: x[0] in canton or x[1] in canton)]
print(len(HKname_v1))

554786
554786


In [10]:
MLname = onename[onename.apply(lambda x: x[0] in pinyin)]

In [33]:
import numpy as np
blur = np.intersect1d(HKname_v1.index, MLname.index)
firstpinyin = HKname_v1[blur]

In [21]:
possiblepinyin = firstpinyin[firstpinyin.apply(lambda x: x[1] in pinyin if len(x)==2 else x[1] in pinyin and x[2] in pinyin)]

In [34]:
MLname[~MLname.index.isin(blur)]

80                                        [HU, JACHIN]
102                                    [ZOU, ZHONGHUA]
198                                       [YANG, XIAO]
275                                    [ZHENG, JINLIN]
278                                  [MOU, CHIA, CHUN]
                              ...                     
2260692                                [HUANG, JIANRU]
2260732                                  [LAO, I, MUI]
2260753    [LIAO, SHIU, HSING, LUCY, "ADMINISTRATRIX"]
2260848                         [ZHANG, MIN, ISABELLE]
2261079                           [HUANG, HSIEH, LIEN]
Name: name, Length: 30175, dtype: object

In [161]:
pinyin_unique = list(set(pinyin) - (set(canton) & set(pinyin)))
canton_unique = list(set(canton) - (set(canton) & set(pinyin)))

In [186]:
def identify(name):
    if len(name)==1: return 'unknown'
    if len(name)>=2:
        if name[0] in last_name_pinyin:
            if name[0] not in last_name_canton:
                return 'ML'
            if name[0] in last_name_canton:
                if name[1] in pinyin_unique:
                    return 'ML'
                elif name[1] in canton:
                    return 'HK'
                elif len(name)>=3:
                    if name[2] in pinyin_unique: return 'HK'
                    elif name[2] in canton_unique: return 'ML'
                else: return 'ML'
                    
        elif name[0] in last_name_canton:
            return 'HK'
        else:
            return 'F'

In [35]:
#Multiple name list
multiname = nameparse[nameparse.apply(lambda x: '&' in x)]

In [180]:
def countname(a):
    count=0
    for item in a:
        if item == '&':
            count+=1
    return count

counting = nameparse.apply(lambda x: countname(x))

In [178]:
def locate(a,n):
    count = 0
    for i,item in enumerate(a):
        if item == '&':
            count += 1
        if count == n:
            return i
    if n == 0:
        return 0
    if n == countname(a):
        return len(a)

In [129]:
def clean(mylist):
    i = 1
    newlist = []
    consecutive = False
    for item in mylist:
        if item == '&':
            if not consecutive:
                newlist.append(item)
                consecutive = True
        else:
            newlist.append(item)
            consecutive = False
    if mylist[-1] == '&':
        mylist = mylist[:-1]
    return newlist

In [130]:
nameparse = nameparse.apply(lambda x: clean(x))

In [159]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r+1) for r in range(len(s)))

outcome = [set(x) for x in powerset({'HK','ML','F','unknown'})]
outcome

[{'ML'},
 {'unknown'},
 {'HK'},
 {'F'},
 {'ML', 'unknown'},
 {'HK', 'ML'},
 {'F', 'ML'},
 {'HK', 'unknown'},
 {'F', 'unknown'},
 {'F', 'HK'},
 {'HK', 'ML', 'unknown'},
 {'F', 'ML', 'unknown'},
 {'F', 'HK', 'ML'},
 {'F', 'HK', 'unknown'},
 {'F', 'HK', 'ML', 'unknown'}]

In [187]:
#This only works on multiple names.
def nameloop(namelist):
    namecount = count(namelist) #Notice that here namecount is actually the number of names - 1.
    origin = []
    for num in range(namecount+1):
        name = namelist[locate(namelist,num):locate(namelist,num+1)]
        if name[0] == '&':
            name = name[1:]
        origin.append(identify(name))
    origin_set = set(origin)
    
    for item in outcome:
        if item == origin_set:
            return item
    #Directly return origin_set may incur problem about order of elements.

In [203]:
import time
start_time = time.process_time() # store the start time

f = lambda x: identify(x) if countname(x)==0 else None
df['tag'] = df['name'].apply(f)
        
mask = df['tag'].isnull()
g = lambda x: nameloop(x) if countname(x)>0 else None
df.loc[mask,'tag'] = df['name'].apply(g)

end_time = time.process_time() # store the end time
print(f"The code took {end_time - start_time} seconds of CPU time to run.")


The code took 26.296685999999966 seconds of CPU time to run.


In [204]:
df.loc[mask,'tag']

1          {HK}
5          {HK}
6          {HK}
8          {HK}
9          {HK}
           ... 
2261120    {HK}
2261121    {HK}
2261122    {HK}
2261123    {HK}
2261125    {HK}
Name: tag, Length: 1239260, dtype: object

In [205]:
df.to_csv('df.csv')